# Forecasting Concurrent Users of PlaiCraft Minecraft Server 

## Introduction

### Relevant backgroud information:

PlaiCraft is a Minecraft server that merges the relms of the popular videogame Minecraft and AI research; they collect data to train and better its model programming. As participants of the research and DSCI 100 students, two varying data sets collected from previous participants were provided to us. In this project, we will utilize the knowledge and skills  we learned from this course as students in a real-world application. Upon analysis of the data sets, the following research question poses paths for conceptual application.

  

### Research Question: 
Can the week and the time of day be used to accrately predict the number of concurrent players online using KNN regression?

### Dataset Description



`sessions.csv` contains a list of all playing sessions from April 6 2024 to September 26 2024. There are 1535 observations representing 1535 playing sessions. There are 5 variables:
- `hashedEmail`(Character): The email that the player inputted when registering. This hashed email can help associated user information on `players.csv`
- `start_time`(Character): The start time of the session, in the format dd/mm/yyyy hh:ss 
- `end_time`(Character): The end time of the session, in the format dd/mm/yyyy hh:ss 
- `original_start_time`(Double): The Unix time of the beginning of the playing session, measured in seconds and to 6 significant figures of precision ($\pm$ 115 days).
- `original_end_time`(Double): The Unix time of the beginning of the playing session, measured in seconds and to 6 significant figures of precision ($\pm$ 115 days).



## Methods & Results

describe the methods you used to perform your analysis from beginning to end that narrates the analysis code.
your report should include code which:

- loads data 

- wrangles and cleans the data to the format necessary for the planned analysis

- performs a summary of the data set that is relevant for exploratory data analysis related to the planned analysis

- creates a visualization of the dataset that is relevant for exploratory data analysis related to the planned analysis

- performs the data analysis

- creates a visualization of the analysis 
## note: all figures should have a figure number and a legend!!!!


Loading

Wrangling

Summary

Visulization of Summary

Modeling

Visulization of Modeling

## Discussion

summarize what you found

-discuss whether this is what you expected to find?

-discuss what impact could such findings have?

-discuss what future questions could this lead to?

### Summary:

Could this data have been better?

## References